In [108]:
from sympy import *
x,y = symbols('x y')
f = Function('f')(x)

# two linear operators
#
# A1 corresponds to   Af = xf'
# A2 corresponds to   Af = (x+1)f'
def A1(f):
    return x * f.diff(x,1)

def A2(f):
    return (x+1) * f.diff(x,1)

# C computes the iterated function composition
#
# ( A o A o ... o A o f)
#   \............../
#    n times
#
def C(A,f,n):
    res = f
    for i in range(n):
        res = expand(A(res))
    return res

# convert f(x) (d/dx)^k -> y^k
#
# this function just searches for all derivatives f^(k)(x) and replaces them with y**k.
# the resulting expression is easier to read and handle.
def T(A):
    res = A
    if isinstance(A,Add):
        for u in A.args:
            u1 = u.args
            for i1 in range(len(u1)):
                if isinstance(u1[i1],Derivative):
                    u2 = u1[i1].args
                    if u2[0]==f:
                        k = u2[1][1]
                        res = res.replace(f.diff(x,k),y**k)
    elif isinstance(A,Mul):
        # in this branch we only have one term
        # the +1-1 helps to reuse the Add branch
        res = T(A+1)-1
    return collect(res,y)


for n in range(8):
    An = C(A2,f,n)
    display(T(An))
    print("---")


f(x)

---


y*(x + 1)

---


y**2*(x**2 + 2*x + 1) + y*(x + 1)

...

---


y**6*(x**6 + 6*x**5 + 15*x**4 + 20*x**3 + 15*x**2 + 6*x + 1) + y**5*(15*x**5 + 75*x**4 + 150*x**3 + 150*x**2 + 75*x + 15) + y**4*(65*x**4 + 260*x**3 + 390*x**2 + 260*x + 65) + y**3*(90*x**3 + 270*x**2 + 270*x + 90) + y**2*(31*x**2 + 62*x + 31) + y*(x + 1)

---


y**7*(x**7 + 7*x**6 + 21*x**5 + 35*x**4 + 35*x**3 + 21*x**2 + 7*x + 1) + y**6*(21*x**6 + 126*x**5 + 315*x**4 + 420*x**3 + 315*x**2 + 126*x + 21) + y**5*(140*x**5 + 700*x**4 + 1400*x**3 + 1400*x**2 + 700*x + 140) + y**4*(350*x**4 + 1400*x**3 + 2100*x**2 + 1400*x + 350) + y**3*(301*x**3 + 903*x**2 + 903*x + 301) + y**2*(63*x**2 + 126*x + 63) + y*(x + 1)

---
